We begin by importing Numpy and PyTorch:

In [0]:
import torch
import numpy as np

##Training data
The training data can be represented using 2 matrices: inputs and targets, each with one row per observation, and one column per variable.

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')


targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [3]:
# Convert inputs and targets to tensors
inputs  = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [4]:
# wieghts and baiess
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[ 0.6216, -0.3608, -0.3145],
        [-1.4892, -0.1314, -1.1788]], requires_grad=True)
tensor([2.4526, 2.1662], requires_grad=True)


In [0]:
def model(x):
  return x @ w.t() + b

In [0]:
predicts = model(inputs)

In [7]:
predicts

tensor([[  10.1301, -166.0344],
        [   7.1369, -220.3533],
        [ -10.0611, -213.3669],
        [  38.7041, -198.9948],
        [ -11.3123, -195.7151]], grad_fn=<AddBackward0>)

In [8]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


##Loss function
Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

* Calculate the difference between the two matrices (preds and targets).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.
The result is a single number, known as the mean squared error (MSE).

In [0]:
def mse(t1 , t2):
  diff = t1- t2
  #deff = deff * deff

  return torch.sum(diff * diff) / diff.numel()



In [10]:
loss = mse(predicts , targets)
loss

tensor(47125.2305, grad_fn=<DivBackward0>)

In [0]:
loss.backward()

In [12]:
print(w)
print(w.grad)
print("#" * 40)
print(b)
print(b.grad)

tensor([[ 0.6216, -0.3608, -0.3145],
        [-1.4892, -0.1314, -1.1788]], requires_grad=True)
tensor([[ -5496.4199,  -7424.6270,  -4313.8008],
        [-24478.8789, -26173.3965, -16313.4473]])
########################################
tensor([2.4526, 2.1662], requires_grad=True)
tensor([ -69.2805, -290.8929])


In [13]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


##Adjust weights and biases using gradient descent
We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

1. Calculate the loss

1. Compute gradients w.r.t the weights and biases

1. Adjust the weights by subtracting a small quantity proportional to the gradient

1. Reset the gradients to zero

Let's implement the above step by step.

In [14]:
preds = model(inputs)
preds

tensor([[  10.1301, -166.0344],
        [   7.1369, -220.3533],
        [ -10.0611, -213.3669],
        [  38.7041, -198.9948],
        [ -11.3123, -195.7151]], grad_fn=<AddBackward0>)

In [15]:
loss = mse(preds,targets)
print(loss)

tensor(47125.2305, grad_fn=<DivBackward0>)


In [16]:
#compute the gradient
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -5496.4199,  -7424.6270,  -4313.8008],
        [-24478.8789, -26173.3965, -16313.4473]])
tensor([ -69.2805, -290.8929])


In [0]:
#Adjust the weights by subtracting a small quantity proportional to the gradien

with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()

In [18]:
print(w)
print(b)

tensor([[ 0.6766, -0.2866, -0.2714],
        [-1.2444,  0.1304, -1.0157]], requires_grad=True)
tensor([2.4533, 2.1691], requires_grad=True)


In [19]:
# With the new weights and biases, the model should have lower loss.
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(32061.5098, grad_fn=<DivBackward0>)


##Train for multiple epochs
To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [0]:
for i in range(300):
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()

  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [21]:
print(loss)

tensor(77.3464, grad_fn=<DivBackward0>)


In [22]:
print(preds)
print(targets)

tensor([[ 59.9304,  71.3364],
        [ 79.8904,  93.6404],
        [119.5237, 147.3016],
        [ 34.3505,  40.0848],
        [ 90.6301, 105.5478]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression using PyTorch built-ins


The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [0]:
import torch.nn as nn

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs  = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

##Dataset and DataLoader
We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [25]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(inputs, targets)
print(train_ds[:3])

(tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.]]))


The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [0]:
from torch.utils.data import DataLoader
batch = 5
train_dl = DataLoader(train_ds, batch , shuffle=True )

In [27]:
for xb , yb in train_dl:
  print(xb)
  print(yb)

tensor([[ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.]])
tensor([[119., 133.],
        [ 56.,  70.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 56.,  70.]])
tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.]])
tensor([[ 22.,  37.],
        [ 81., 101.],
        [103., 119.],
        [103., 119.],
        [119., 133.]])
tensor([[ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.]])
tensor([[103., 119.],
        [ 22.,  37.],
        [ 81., 101.],
        [ 81., 101.],
        [119., 133.]])


##nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [28]:
# Define model 
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)


Parameter containing:
tensor([[-0.4605, -0.1215,  0.1973],
        [-0.1896,  0.5283, -0.3825]], requires_grad=True)
Parameter containing:
tensor([-0.1227, -0.0360], requires_grad=True)


In [29]:
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.4605, -0.1215,  0.1973],
        [-0.1896,  0.5283, -0.3825]], requires_grad=True), Parameter containing:
tensor([-0.1227, -0.0360], requires_grad=True)]


In [30]:
preds= model(inputs)
preds

tensor([[-33.3896,   5.0722],
        [-40.0849,   4.7212],
        [-45.0139,  32.0783],
        [-45.0121, -10.8118],
        [-29.7423,  10.8245],
        [-33.3896,   5.0722],
        [-40.0849,   4.7212],
        [-45.0139,  32.0783],
        [-45.0121, -10.8118],
        [-29.7423,  10.8245],
        [-33.3896,   5.0722],
        [-40.0849,   4.7212],
        [-45.0139,  32.0783],
        [-45.0121, -10.8118],
        [-29.7423,  10.8245]], grad_fn=<AddmmBackward>)

In [31]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Loss Function

In [0]:
import torch.nn.functional as f
loss_fn = f.mse_loss


In [33]:
loss = loss_fn(preds, targets)
print(loss)

tensor(10932.2080, grad_fn=<MseLossBackward>)


##Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD stands for stochastic gradient descent. It is called stochastic because samples are selected in batches (often with random shuffling) instead of as a single group.

In [0]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr = 1e-5) 

##Train the model
We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

1. Calculate the loss

1. Compute gradients w.r.t the weights and biases

1. Adjust the weights by subtracting a small quantity proportional to the gradient

1. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function fit which trains the model for a given number of epochs.

In [0]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [37]:
fit(200, model, loss_fn, opt,train_dl)

Epoch [10/200], Loss: 115.3670
Epoch [20/200], Loss: 94.2249
Epoch [30/200], Loss: 134.6416
Epoch [40/200], Loss: 80.8137
Epoch [50/200], Loss: 55.9508
Epoch [60/200], Loss: 61.6654
Epoch [70/200], Loss: 31.9624
Epoch [80/200], Loss: 62.2898
Epoch [90/200], Loss: 38.9312
Epoch [100/200], Loss: 48.6849
Epoch [110/200], Loss: 30.0201
Epoch [120/200], Loss: 24.6369
Epoch [130/200], Loss: 26.8217
Epoch [140/200], Loss: 14.1122
Epoch [150/200], Loss: 23.1439
Epoch [160/200], Loss: 32.7236
Epoch [170/200], Loss: 31.9469
Epoch [180/200], Loss: 26.3564
Epoch [190/200], Loss: 18.8793
Epoch [200/200], Loss: 21.7486


generate predictions using our model and verify that they're close to our targets.

In [38]:
preds = model(inputs)
preds

tensor([[ 57.3136,  71.1518],
        [ 81.9933,  96.8169],
        [119.0840, 142.0462],
        [ 21.9149,  40.1657],
        [101.0284, 110.4682],
        [ 57.3136,  71.1518],
        [ 81.9933,  96.8169],
        [119.0840, 142.0462],
        [ 21.9149,  40.1657],
        [101.0284, 110.4682],
        [ 57.3136,  71.1518],
        [ 81.9933,  96.8169],
        [119.0840, 142.0462],
        [ 21.9149,  40.1657],
        [101.0284, 110.4682]], grad_fn=<AddmmBackward>)

#### Compare with targets


In [39]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Indeed, the predictions are quite close to our targets, and now we have a fairly good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall and humidity in a region.